# Market Basket Text Analysis of One Health and Antimicrobial Research Papers

## Introduction

### What is Market Basket Analysis?

#### The Problem

Too long; Didn't Read: We're looking for items or words that appear commonly together.

Consider a grocery store. Over the course of a month, **1000s** of people while have stumbled through our example grocery store. Some customers, like my parents, may pile up shopping carts to the cieling with microwave dinners. Customers a little more like me, or the average college student, might walk out with fewer than a dozen items. 

Consider how many different items exist in any given grocery store. Ghetto Kroger probably has several hundred items while Gucci Kroger has maybe more than a thousand? 

Out of all the possible things we could buy in a grocery store, customers end up only buying a select few. On top of that, we could reasonably guess that most purchases are biased towards a few items. I buy a half-gallon of milk, a loaf of bread, an a box of Lucky Charms when I go to the grocery store.

Finding the most popular item is trivial. We can just keep a running count for each unique item and use a bar chart for visualization. Finding the most popular combination of items out of **every possible combination** is not so trivial. 

Think about the dataframe that would hold all of this. We'd have a column for every item and a row for every receipt. This dataframe could easily get to hundres of thousands of observations for several thousand variables in any decent suburb. Additionally, this matrix also tends to be sparse, meaning that almost entries are '0' or empty.

#### The Solution

Enter Association Rules or Market Basket Analysis. This tool uses a few simple steps to find probabilites that: given 'item a' ('on the left', 'leftside') is purchased, 'item b' ('on the right', 'rightside') is also purchased.

Mathematically, let $X$, $Y$ be itemsets and $T$ be the set of all transactions taken place. Then an association rule looks simply like, 
$$ X \Rightarrow Y $$

Three terms we'll need to define:

Too long; Didn't Read: Use a couple rules to find probably associated items in a dataset.

1. Support - How many times the leftside appears in the dataset
$$ supp(X) =  \frac{|t \in T; X \in t|}{|T|}$$

2. Confidence - The proportion of all transactions that contain both the left and right side. Or, a little more correctly, given a transaction contains the left side ($X$) the transaction also contains the right side ($Y$)
$$ conf (X \Rightarrow Y) = \frac{supp(X \cup Y)}{supp(X)} $$

My Latex is not quite pro, yet. My $f$ in the above looks a little whack, and I can't figure it out.

3. Lift - The ratio of a given rule compared to what would be expected if th left and right sides truly appeared independent of one another.
$$ lift(X \Rightarrow Y) = \frac{supp(X \cup Y)}{supp(X) \times supp(Y)} $$

These are 'measures of interestingness'. There are others, but, in my mind, these are 'the big three'.

Finding Association Rules is a two step process:

1. Set some minimum support threshold to find frequent itemsets in the transaction database. Itemsets with less than this support are thrown out.
2. Set some minimum confidence threshold to these supported itemsets in order to form rules. Rules with less than this confidence are thrown out.

Once we have a few itemsets, the second step seems trivial to apply. The first step, at a glance, may seem less trivial. The first step seems to involve searching through every possible combination of itemsets to meet our given support threshold. Well, in short, the support rule has a property ('downward-closure') which, for us, boils down to 'unsupported itemsets can't be contained in larger supported itemsets'. This property is what algorithms like [Apriori](http://rakesh.agrawal-family.com/papers/vldb94apriori.pdf) take advantage of to efficiently search these super large, sparse transaction databases.

Lift is not directly used in the process mentioned above. Instead, lift really tells us what a given rule ($ X \Rightarrow Y$) *means*. If the reported lift is:

1. Equal to one - There is no relation between $X$ and $Y$. People buying cereal may or may not buy bread.
2. Greater than one - $X$ and $Y$ seem to be dependent on another. When people buy milk, they usually also buy cereal.
3. Less than one - Either itemset in this rule has a negative impact on the presence of the other itemset. People buying cereal don't seem to buy hot sauce.

### The Inspiration for MBA in AMR

I thought AMR would be a good use case for this method because words (or $n$-gram sequences) could potentially be an equivalent scenario to a grocery store. Instead of milk, we have 'antimicrobial resistance.' And instead of receipts, we have research papers where the words potentially appear. 

I also think Market Basket Analysis is, essentially, the easiest Machine Learning technique for us sutdents to get down. Our machine thinks everything is associated to start with. Then, as we increase required support and confidence, we learn which itemsets are (and are not) associated with eachother. We could do this pretty naively, but, utilizing support's downward-closure gives us a nice performance bump to handle the usual use-case.

I may be slightly concerned that our use case may not be large enough. I think only 4-ish thousand papers had associated PDFs that I then cleaned up. We may want to go back at this with a web scraper to find the other 20k papers that were found when searching with `getpapers`. 

I think I read somewhere that I need at least a few hundred supporting observations before a rule sould be considered statistically significant. And several hundred is not too small compared to 4,000 (in the general field of data science, I think). I'll have to keep this in mind when working on this.

## The Code 

To get association rules for our text data set, I'll use the [apyori](https://pypi.org/project/apyori/) package for it's Apriori implementation. (I believe a similar implementation also exists in the [mlxtend](https://pypi.org/project/mlxtend/) package.) 

This implentation is looking for a 'list of lists' to operate on, so I'll need to read in each clean text file. Then I'll create list containing an entry for each gram in the file.




In [2]:
# Imports -------------------------------------------------------
import pandas as pd

import nltk
from nltk import ngrams
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')


from pathlib import Path
from random import sample # Used to create smaller datasets for testing
import sys
import json


[nltk_data] Downloading package wordnet to /home/tanner/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# Import searched_terms
# I know there's a better way to do this by appending to the path
# sys.path.append('../src/')
# But I couln't figure it out
searched_terms = []
with open('src/json/searched_terms.json', 'r') as searched_terms_file:
    # Load in the json and pull out the array from the loaded dictionary
    searched_terms = json.load(searched_terms_file)["searched_terms"]

# Lemmatize the searched terms
# This puts our searched terms through the same relevent cleaning as the cleaned texts
# Maximizing potential findings
lemmatizer = WordNetLemmatizer()
for i, term in enumerate(searched_terms):
    # Split the given term on space then lemmatize the word and rejoin with a space
    cleaned_words = ' '.join([lemmatizer.lemmatize(word) for word in term.lower().split(' ')])
    searched_terms[i] = cleaned_words

# Set my working directory
working_directory = Path('AmsContainer')

# Glob (i.e. search) that driectory for cleaned files
cleaned_text_paths = list(working_directory.glob('**/cleaned_output.txt'))

word_counts = []

for cleaned_text_path in cleaned_text_paths:
    with open(cleaned_text_path, 'r') as cleaned_text_file:
        # Read the file and create bigrams
        text = cleaned_text_file.read()
        bigrams = ngrams(word_tokenize(text), 2)
        

        # The bigrams produced are intitially in the format:
        # ('word_1', 'word_2'), but I'd like to work with:
        # 'word_1 word_2' instead
        # 'word_1 word_2' matches the format of searched_terms
        bigrams = (' '.join(gram) for gram in bigrams)
        
        # This is too loose, need to figure out a better way to count
        local_counts = []
        for term in searched_terms:
            if term in text:
                local_counts.append(1)
            else:
                local_counts.append(0)
        word_counts.append(local_counts)
        #word_counts.append([1 if term in gram else 0 for term in searched_terms])
        

# This commented block was my original way of using 
# every unique word as a header (a potential item in the MBA).
# I think this is a little overkill, so I just used our searched
# terms as headers in a dataframe

# for cleaned_text_path in cleaned_text_paths:
#    with open(cleaned_text_path, 'r') as cleaned_text_file:
#        text = cleaned_text_file.read() # Read in the file

        # Create a list of grams (i.e. words) 
        # by splitting the file on whitespace
#        tokenized_text = { *text.split(' ') } 

        # Store the tokenized text into the transactions list
#        headers = headers.union(sample(tokenized_text, 100))

# headers = [ *headers ]
# word_counts = []

# for cleaned_text_path in cleaned_text_paths:
#    with open(cleaned_text_path, 'r') as cleaned_text_file:
#        tokenized_text = cleaned_text_file.read().split(' ')
#        local_counts = []
#        for header in headers: 
#            if header in tokenized_text:
#                local_counts.append(1)
#            else:
#                local_counts.append(0)
#        word_counts.append(local_counts)

word_transactions = pd.DataFrame(word_counts, columns = searched_terms)


environment
Found drug resistance
Found multidrug resistance
Found resistance
Found arb
Found ar
Found fish
Found human
Found drug resistance
Found resistance
Found ar
Found mdr
Found fish
Found bovine
Found human
Found resistance
Found ar
Found fish
Found bovine
Found animal
Found human
Found environment
Found drug resistance
Found resistance
Found arb
Found ar
Found fish
Found bovine
Found pharmaceutical
Found animal
Found human
Found environment
Found ecosystem
Found antibiotic resistance
Found resistance
Found arb
Found ar
Found pig
Found agriculture
Found drinking water
Found human
Found environment
Found drug resistance
Found resistance
Found arb
Found ar
Found cow
Found fish
Found bovine
Found pharmaceutical
Found human
Found environment
Found resistance
Found arb
Found ar
Found chicken
Found pig
Found pharmaceutical
Found human
Found environment
Found resistance
Found arb
Found ar
Found fish
Found bovine
Found animal
Found human
Found environment
Found ecosystem
Found drug resi

In [10]:
print(searched_terms)
for i, value in enumerate(word_transactions.loc[:,'beef']):
    if value == 1:
        print(i)

['one health', 'one medicine', 'animal', 'human', 'environment', 'ecosystem', 'ecohealth', 'antimicrobial resistance', 'antibiotic resistance', 'drug resistance', 'multidrug resistance', 'resistance', 'amr', 'arb', 'ar', 'mdr', 'dairy', 'cow', 'beef', 'cattle', 'poultry', 'swine', 'chicken', 'pig', 'turkey', 'fish', 'porcine', 'bovine', 'soil', 'agriculture', 'wastewater', 'pharmaceutical', 'drinking water', 'groundwater', 'surface water', 'compost', 'manure', 'biosolids', 'aquaculture']
1
8
17
18
19
20
21
23
24
25
28
29
41
42
43
44
45
46
48
53
68
88
96
99
101
104
108
109
115
124
125
141
152
153
161
162
163
166
168
170
171
174
178
180
210
212
217
235
242
246
251
258
277
281
284
292
295
336
341
360
365
367
385
391
393
399
400
408
418
423
443
446
451
453
456
460
464
470
475
477
481
492
504
507
526
535
540
555
557
565
567
572
575
579
584
585
587
589
599
608
618
624
625
638
641
649
650
651
652
659
666
667
686
691
692
698
700
719
729
742
750
752
760
777
780
785
786
787
795
798
799
804
805
8

Now that we have our 'list of lists,' we'll run this transaction list through `apyori`'s apriori implementation.

In [2]:
from apyori import apriori
#from mlxtend.frequent_patterns import apriori
#from mlxtend.frequent_patterns import association_rules

# I did say this was 'fast' and that 'we may not have enough data.' 
# I still stand by that, but this does seem to take a long time.
# I think the word count is still pretty high even after cleaning.
rules = apriori(transactions, 
                     min_support = 0.02, 
                     min_confidence = 0.80, 
                     min_lift = 1.0, 
                     max_length = 100)

# Output the first five rules
#
# This looks weird, but I'm trying 
# to take advantage of the generator
# returned by apriori() to save some memory
[rule for i, rule in enumerate(rules) if i < 5]

assigned


In [2]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_word_sets = apriori(word_transactions, 
                             min_support = 0.02,
                             use_colnames = True)

rules = association_rules(frequent_word_Sets, 
                          metric = "lift", 
                          min_threshold = 1)

rules.head()

## Visualizations

Because every good notebook needs a few graphs.

These associations (when results come back) would be really well represented by a network chart. I want to try creating one of these using [plot.ly](https://plot.ly/python/network-graphs/).



In [ ]:
import plotly.graph_objects as go
import networkx as networkx

